In [ ]:
!pip install -q tensorflow tensorflow_io


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving guess.zip to guess.zip


In [ ]:
!rm -f speech_commands_v0.02.tar.gz
!rm -rf speech_commands


In [ ]:
!wget http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz


--2025-05-04 19:03:27--  http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.206.207, 142.251.184.207, 64.233.179.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.206.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428923189 (2.3G) [application/gzip]
Saving to: ‘speech_commands_v0.02.tar.gz’

speech_commands_v0. 100%[===================>]   2.26G  95.8MB/s    in 14s     

2025-05-04 19:03:41 (160 MB/s) - ‘speech_commands_v0.02.tar.gz’ saved [2428923189/2428923189]



In [ ]:
# Step 1: Clean up old files/folders
!rm -rf speech_commands dataset
!mkdir -p dataset/no

# Step 2: Extract the entire archive
!mkdir -p speech_commands
!tar -xzf speech_commands_v0.02.tar.gz -C speech_commands

# Step 3: Copy only "no" samples into dataset
!cp speech_commands/no/*.wav dataset/no/

# Step 4: Confirm copy worked
!ls dataset/no | head -10


012c8314_nohash_0.wav
0132a06d_nohash_0.wav
0132a06d_nohash_1.wav
0132a06d_nohash_2.wav
0132a06d_nohash_3.wav
0132a06d_nohash_4.wav
0137b3f4_nohash_0.wav
0137b3f4_nohash_1.wav
0137b3f4_nohash_2.wav
0137b3f4_nohash_3.wav


In [ ]:
import zipfile

# Path to your uploaded file (adjust if needed)
zip_path = "/content/guess.zip"
extract_path = "/content/dataset/guess"

# Unzip into dataset/guess
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify the result
import os
print("Extracted files:", len(os.listdir(extract_path)))
os.listdir(extract_path)[:10]


Extracted files: 2


['guess', '__MACOSX']

In [ ]:
import shutil
import glob

# Move WAV files from nested folder to dataset/guess
nested_path = "/content/dataset/guess/guess/*.wav"
target_path = "/content/dataset/guess/"

for wav_file in glob.glob(nested_path):
    shutil.move(wav_file, target_path)

# Remove extra folders
shutil.rmtree("/content/dataset/guess/guess", ignore_errors=True)
shutil.rmtree("/content/dataset/guess/__MACOSX", ignore_errors=True)

# Confirm final structure
import os
print("Final WAV files:", len(os.listdir(target_path)))
os.listdir(target_path)[:10]


Final WAV files: 0


[]

In [ ]:
import os

for root, dirs, files in os.walk("/content"):
    for name in files:
        if name.endswith(".wav"):
            print(os.path.join(root, name))


Streaming output truncated to the last 5000 lines.
/content/speech_commands/right/e63b69c5_nohash_0.wav
/content/speech_commands/right/d8ed3745_nohash_0.wav
/content/speech_commands/right/a045368c_nohash_0.wav
/content/speech_commands/right/d71ad617_nohash_0.wav
/content/speech_commands/right/dfb6450b_nohash_0.wav
/content/speech_commands/right/2579e514_nohash_0.wav
/content/speech_commands/right/86f12ac0_nohash_1.wav
/content/speech_commands/right/0f2442b6_nohash_0.wav
/content/speech_commands/right/d0faf7e4_nohash_4.wav
/content/speech_commands/right/23da904f_nohash_3.wav
/content/speech_commands/right/af405b69_nohash_1.wav
/content/speech_commands/right/195c120a_nohash_3.wav
/content/speech_commands/right/fda46b78_nohash_0.wav
/content/speech_commands/right/5a98d407_nohash_2.wav
/content/speech_commands/right/57152045_nohash_4.wav
/content/speech_commands/right/7e1054e7_nohash_0.wav
/content/speech_commands/right/674ca5ea_nohash_1.wav
/content/speech_commands/right/c44d2a58_nohash_0

In [ ]:
!unzip -q /content/guess.zip -d /content/speech_commands/


In [ ]:
!unzip -q /content/guess.zip -d /content/speech_commands/guess/


In [ ]:
!unzip -l /content/guess.zip | head -20


Archive:  /content/guess.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
        0  2025-05-04 18:13   guess/
     9596  2025-05-04 18:09   guess/New Recording 48.m4a
      436  2025-05-04 18:09   __MACOSX/guess/._New Recording 48.m4a
    11661  2025-05-04 18:09   guess/New Recording 49.m4a
      436  2025-05-04 18:09   __MACOSX/guess/._New Recording 49.m4a
     9998  2025-05-04 18:09   guess/New Recording 39.m4a
      436  2025-05-04 18:09   __MACOSX/guess/._New Recording 39.m4a
     9328  2025-05-04 18:09   guess/New Recording 38.m4a
      436  2025-05-04 18:09   __MACOSX/guess/._New Recording 38.m4a
     8966  2025-05-04 18:09   guess/New Recording 53.m4a
      436  2025-05-04 18:09   __MACOSX/guess/._New Recording 53.m4a
    10388  2025-05-04 18:09   guess/New Recording 47.m4a
      436  2025-05-04 18:09   __MACOSX/guess/._New Recording 47.m4a
     8660  2025-05-04 18:09   guess/New Recording 46.m4a
      436  2025-05-04 18:09   __MACOSX/guess/._New Record

In [ ]:
# 1. Unzip into a temp directory
!unzip -q /content/guess.zip -d /content/tmp_guess

# 2. Create destination folder for WAVs
!mkdir -p /content/speech_commands/guess_wav

# 3. Install ffmpeg (if not already installed)
!apt-get -y install ffmpeg

# 4. Convert all .m4a to .wav
import os
import subprocess

m4a_dir = "/content/tmp_guess/guess"
wav_dir = "/content/speech_commands/guess_wav"

for fname in os.listdir(m4a_dir):
    if fname.endswith(".m4a"):
        m4a_path = os.path.join(m4a_dir, fname)
        wav_path = os.path.join(wav_dir, fname.replace(".m4a", ".wav"))
        subprocess.run(["ffmpeg", "-y", "-i", m4a_path, wav_path],
                       stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

print("Conversion complete. WAV files:", len(os.listdir(wav_dir)))


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Conversion complete. WAV files: 20


In [ ]:
ls speech_commands/

_background_noise_/  follow/   LICENSE    right/            up/
backward/            forward/  __MACOSX/  seven/            validation_list.txt
bed/                 four/     marvin/    sheila/           visual/
bird/                go/       nine/      six/              wow/
cat/                 guess/    no/        stop/             yes/
dog/                 happy/    off/       testing_list.txt  zero/
down/                house/    on/        three/
eight/               learn/    one/       tree/
five/                left/     README.md  two/


In [ ]:
ls dataset/

guess/  no/


In [ ]:
!rm -rf /content/guess_wav /content/dataset/guess_wav /content/speech_commands/guess_raw


In [ ]:
import os
import subprocess

# Unzip the raw recordings
!unzip -q /content/guess.zip -d /content/speech_commands/guess_raw

# Set source and destination directories
src_dir = "/content/speech_commands/guess_raw/guess"
dst_dir = "/content/guess_wav"
os.makedirs(dst_dir, exist_ok=True)

# Convert .m4a to .wav
converted = 0
for file in os.listdir(src_dir):
    if file.endswith(".m4a"):
        in_path = os.path.join(src_dir, file)
        out_path = os.path.join(dst_dir, file.replace(".m4a", ".wav"))
        subprocess.run(["ffmpeg", "-i", in_path, out_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        converted += 1

print(f" Converted {converted} .m4a files to .wav in /content/guess_wav/")


✅ Converted 20 .m4a files to .wav in /content/guess_wav/


In [ ]:
ls speech_commands/guess_raw/guess

'New Recording 38.m4a'  'New Recording 45.m4a'  'New Recording 52.m4a'
'New Recording 39.m4a'  'New Recording 46.m4a'  'New Recording 53.m4a'
'New Recording 40.m4a'  'New Recording 47.m4a'  'New Recording 54.m4a'
'New Recording 41.m4a'  'New Recording 48.m4a'  'New Recording 55.m4a'
'New Recording 42.m4a'  'New Recording 49.m4a'  'New Recording 56.m4a'
'New Recording 43.m4a'  'New Recording 50.m4a'  'New Recording 57.m4a'
'New Recording 44.m4a'  'New Recording 51.m4a'


In [ ]:
import shutil

os.makedirs("/content/dataset", exist_ok=True)
shutil.move("/content/guess_wav", "/content/dataset/guess_wav")

print(" Moved guess_wav to /content/dataset/guess_wav/")


✅ Moved guess_wav to /content/dataset/guess_wav/


In [ ]:
import os

guess_dir = "/content/dataset/guess_wav"
no_dir = "/content/dataset/no"

guess_count = len([f for f in os.listdir(guess_dir) if f.endswith('.wav')])
no_count = len([f for f in os.listdir(no_dir) if f.endswith('.wav')])

print(f" 'guess_wav' samples: {guess_count}")
print(f" 'no' samples: {no_count}")

diff = no_count - guess_count
if diff > 0:
    print(f" Need to duplicate {diff} 'guess_wav' samples to match 'no'")
else:
    print(f" 'guess_wav' already has enough or more samples than 'no' (difference = {diff})")


🎯 'guess_wav' samples: 20
🎯 'no' samples: 3941
🔁 Need to duplicate 3921 'guess_wav' samples to match 'no'


In [ ]:
import os
import shutil

src_dir = "/content/dataset/guess_wav"
files = [f for f in os.listdir(src_dir) if f.endswith('.wav')]
target_count = 3941  # To match 'no' count
current_count = len(files)

print(f" Duplicating {target_count - current_count} samples...")

for i in range(target_count - current_count):
    original = os.path.join(src_dir, files[i % current_count])
    duplicate_name = f"dup_{i}_{os.path.basename(original)}"
    duplicate_path = os.path.join(src_dir, duplicate_name)
    shutil.copy(original, duplicate_path)

print(f" Done! Total 'guess_wav' samples: {len(os.listdir(src_dir))}")


🔁 Duplicating 3921 samples...
✅ Done! Total 'guess_wav' samples: 3941


In [ ]:
import shutil
import os

base_dir = "/content/speech_commands_subset"
os.makedirs(base_dir, exist_ok=True)

# Define new paths
new_guess = os.path.join(base_dir, "guess")
new_no = os.path.join(base_dir, "no")

# Recreate and copy files
if os.path.exists(new_guess):
    shutil.rmtree(new_guess)
shutil.copytree("/content/dataset/guess_wav", new_guess)

if os.path.exists(new_no):
    shutil.rmtree(new_no)
shutil.copytree("/content/dataset/no", new_no)

print(f" Dataset prepared at: {base_dir}")
print(f" guess samples: {len(os.listdir(new_guess))}")
print(f" no samples: {len(os.listdir(new_no))}")


✅ Dataset prepared at: /content/speech_commands_subset
🎧 guess samples: 3941
🛑 no samples: 3941


In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
from tensorflow.signal import mfccs_from_log_mel_spectrograms

# Parameters
SAMPLE_RATE = 16000
DURATION = 1  # in seconds
NUM_MFCC = 13

# Directories
data_dir = "/content/speech_commands_subset"
labels = ["guess", "no"]

def load_wav_file(filepath):
    audio = tf.io.read_file(filepath)
    audio, _ = tf.audio.decode_wav(audio, desired_channels=1, desired_samples=SAMPLE_RATE)
    return tf.squeeze(audio, axis=-1)

def extract_mfcc(audio):
    stft = tf.signal.stft(audio, frame_length=256, frame_step=128)
    spectrogram = tf.abs(stft)
    mel = tf.signal.linear_to_mel_weight_matrix(40, stft.shape[-1], SAMPLE_RATE, 20, 4000)
    mel_spectrogram = tf.tensordot(spectrogram, mel, 1)
    mel_spectrogram.set_shape(spectrogram.shape[:-1].concatenate(mel.shape[-1:]))
    log_mel = tf.math.log(mel_spectrogram + 1e-6)
    mfccs = mfccs_from_log_mel_spectrograms(log_mel)[..., :NUM_MFCC]
    return mfccs

# Load data
X, y = [], []
for idx, label in enumerate(labels):
    files = tf.io.gfile.glob(os.path.join(data_dir, label, "*.wav"))
    for file in random.sample(files, min(1000, len(files))):  # use up to 1000 samples per class
        audio = load_wav_file(file)
        mfcc = extract_mfcc(audio)
        X.append(mfcc.numpy())
        y.append(idx)

X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', dtype='float32')
X = np.array(X)
y = np.array(y)

# Train/Val split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

print(" Feature extraction complete")
print(" X_train:", X_train.shape)
print(" X_val:", X_val.shape)


✅ Feature extraction complete
📦 X_train: (1600, 124, 13)
📦 X_val: (400, 124, 13)


In [ ]:
import os
import numpy as np
import soundfile as sf
import random
import shutil

# Define paths
DATASET_DIR = "/content/dataset"
SILENCE_DIR = os.path.join(DATASET_DIR, "silence")
UNKNOWN_DIR = os.path.join(DATASET_DIR, "unknown")
SOURCE_NO_DIR = os.path.join(DATASET_DIR, "no")
SAMPLE_RATE = 16000
DURATION = 1  # 1 second
SILENCE_COUNT = 500
UNKNOWN_COUNT = 500

# Create target folders if not exist
os.makedirs(SILENCE_DIR, exist_ok=True)
os.makedirs(UNKNOWN_DIR, exist_ok=True)

# 1. Generate silence: 1-second zero arrays
for i in range(SILENCE_COUNT):
    silence = np.zeros(SAMPLE_RATE, dtype=np.int16)
    sf.write(f"{SILENCE_DIR}/silence_{i}.wav", silence, SAMPLE_RATE)

print(f" Generated {SILENCE_COUNT} silence samples.")

# 2. Generate unknown: by copying random 'no' files
no_files = [f for f in os.listdir(SOURCE_NO_DIR) if f.endswith(".wav")]
random.shuffle(no_files)

for i in range(UNKNOWN_COUNT):
    src_path = os.path.join(SOURCE_NO_DIR, no_files[i % len(no_files)])
    dst_path = os.path.join(UNKNOWN_DIR, f"unknown_{i}.wav")
    shutil.copy(src_path, dst_path)

print(f" Created {UNKNOWN_COUNT} unknown samples from 'no' dataset.")

#  Final dataset structure check
for d in [SILENCE_DIR, UNKNOWN_DIR]:
    print(f"{d}: {len(os.listdir(d))} files")


✅ Generated 500 silence samples.
✅ Created 500 unknown samples from 'no' dataset.
/content/dataset/silence: 500 files
/content/dataset/unknown: 500 files


In [ ]:
!pip install numpy==1.23.5 --quiet


In [ ]:
!pip install librosa==0.9.2


In [ ]:
import os
import librosa
import numpy as np
from tqdm import tqdm

# Directory paths for each class
label_map = {
    "guess_wav": 0,
    "no": 1,
    "silence": 2,
    "unknown": 3
}

X = []
y = []

print(" Extracting MFCC features for all classes...\n")

for folder, label in label_map.items():
    folder_path = f"/content/dataset/{folder}"
    for file_name in tqdm(os.listdir(folder_path), desc=f"Processing '{folder}'"):
        if not file_name.endswith(".wav"):
            continue
        file_path = os.path.join(folder_path, file_name)
        try:
            audio, sr = librosa.load(file_path, sr=16000)
            mfcc = librosa.feature.mfcc(audio, sr=sr, n_mfcc=13)
            if mfcc.shape[1] < 124:
                # Pad if shorter
                mfcc = np.pad(mfcc, ((0, 0), (0, 124 - mfcc.shape[1])), mode='constant')
            else:
                # Trim if longer
                mfcc = mfcc[:, :124]
            X.append(mfcc.T)  # Shape: (124, 13)
            y.append(label)
        except Exception as e:
            print(f"⚠️ Skipping {file_path}: {e}")

X = np.array(X)
y = np.array(y)

print(f"\n Extracted: {X.shape[0]} samples")
print(f" X shape: {X.shape}, y shape: {y.shape}")

# Save arrays
np.save("/content/dataset/X_full.npy", X)
np.save("/content/dataset/y_full.npy", y)

print(" Saved: X_full.npy and y_full.npy")


🔄 Extracting MFCC features for all classes...



Streaming output truncated to the last 5000 lines.
 1.5258789e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(audio, sr=sr, n_mfcc=13)
Processing 'no':  53%|█████▎    | 2082/3941 [00:31<00:30, 60.86it/s]<ipython-input-54-24bf2d8bce02>:27: FutureWarning: Pass y=[-0.00891113 -0.01608276 -0.01669312 ...  0.00793457  0.0090332
  0.00650024] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(audio, sr=sr, n_mfcc=13)
<ipython-input-54-24bf2d8bce02>:27: FutureWarning: Pass y=[9.1552734e-05 9.1552734e-04 1.9531250e-03 ... 1.9226074e-03 1.7395020e-03
 1.1596680e-03] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(audio, sr=sr, n_mfcc=13)
<ipython-input-54-24bf2d8bce02>:27: FutureWarning: Pass y=[ 0.0005188   0.00085449  0.00079346 ... -0.00170898 -0.00125122
 


✅ Extracted: 8882 samples
📦 X shape: (8882, 124, 13), y shape: (8882,)
💾 Saved: X_full.npy and y_full.npy


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf

# Load data
X = np.load('/content/dataset/X_full.npy')  # (8882, 124, 13)
y = np.load('/content/dataset/y_full.npy')  # (8882,)

# One-hot encode labels
y_cat = to_categorical(y, num_classes=4)

# Reshape for Conv2D: (samples, height, width, channels)
X = X[..., np.newaxis]

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y_cat, test_size=0.2, random_state=42, stratify=y)

# Build model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(124, 13, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=64)

# Save model
model.save("keyword_model.h5")
print(" Model saved as 'keyword_model.h5'")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 122, 11, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 61, 5, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 59, 3, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 29, 1, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1856)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       237,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 257,028 (1004.02 KB)

 Trainable params: 257,028 (1004.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
112/112 ━━━━━━━━━━━━━━━━━━━━ 15s 99ms/step - accuracy: 0.7757 - loss: 3.7908 - val_accuracy: 0.9392 - val_loss: 0.2049
Epoch 2/15
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 89ms/step - accuracy: 0.9373 - loss: 0.2025 - val_accuracy: 0.9432 - val_loss: 0.1915
Epoch 3/15
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 89ms/step - accuracy: 0.9397 - loss: 0.1945 - val_accuracy: 0.9437 - val_loss: 0.1948
Epoch 4/15
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 92ms/step - accuracy: 0.9411 - loss: 0.1838 - val_accuracy: 0.9432 - val_loss: 0.1895
Epoch 5/15
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 83ms/step - accuracy: 0.9411 - loss: 0.1835 - val_accuracy: 0.9437 - val_loss: 0.1880
Epoch 6/15
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 89ms/step - accuracy: 0.9437 - loss: 0.1825 - val_accuracy: 0.9437 - val_loss: 0.1888
Epoch 7/15
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 88ms/step - accuracy: 0.9405 - loss: 0.1818 - val_accuracy: 0.9437 - val_loss: 0.1936
Epoch 8/15
112/112 ━━━━━━━━━━━━━━━━━━━━ 9s 77ms/step - accuracy: 0.9425 - loss: 0.1749 - v

✅ Model saved as 'keyword_model.h5'


In [ ]:
import tensorflow as tf

# Load model
model = tf.keras.models.load_model("keyword_model.h5")

# Convert to TFLite (float32 first)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save TFLite model
with open("keyword_model.tflite", "wb") as f:
    f.write(tflite_model)
print(" Saved: keyword_model.tflite (float32)")

# Now quantize to int8
def representative_dataset_gen():
    for i in range(100):
        yield [X_train[i:i+1].astype(np.float32)]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_quant_model = converter.convert()

# Save quantized model
with open("keyword_model_quant.tflite", "wb") as f:
    f.write(tflite_quant_model)
print(" Saved: keyword_model_quant.tflite (int8)")


Saved artifact at '/tmp/tmpxokb6j7q'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 124, 13, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  137110626204560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137110625077968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137110625078736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137110625075664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137110625077008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137110625091408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137110625077392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137110625090256: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Saved: keyword_model.tflite (float32)
Saved artifact at '/tmp/tmp9rcaghri'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ Saved: keyword_model_quant.tflite (int8)


In [ ]:
from google.colab import files
files.download('keyword_model_quant.tflite')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
interpreter = tf.lite.Interpreter(model_path="keyword_model_quant.tflite")


In [ ]:
!xxd -i keyword_model_quant.tflite > model.cpp


In [ ]:
# Convert TFLite model to C array
!xxd -i keyword_model_quant.tflite > model.cpp

# Fix the array name and wrap with C++ header includes
with open("model.cpp", "r") as f:
    lines = f.readlines()

# Replace variable name and wrap in header
with open("model.cpp", "w") as f:
    f.write('#include "model.h"\n\n')
    for line in lines:
        f.write(line.replace('unsigned char keyword_model_quant_tflite[]', 'const unsigned char g_model[]')
                  .replace('unsigned int keyword_model_quant_tflite_len', 'const int g_model_len'))

# Automatically download in Colab
from google.colab import files
files.download("model.cpp")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving New Recording 56.m4a to New Recording 56.m4a


In [ ]:
import os

# Convert all uploaded .m4a to .wav
for m4a_file in uploaded.keys():
    if m4a_file.endswith('.m4a'):
        wav_file = m4a_file.replace('.m4a', '.wav')
        os.system(f"ffmpeg -i '{m4a_file}' -ar 16000 -ac 1 '{wav_file}'")
        print(f"Converted {m4a_file} → {wav_file}")


Converted New Recording 56.m4a → New Recording 56.wav


In [ ]:
import numpy as np
import librosa

def extract_mfcc(filepath):
    signal, sr = librosa.load(filepath, sr=16000)
    mfcc = librosa.feature.mfcc(signal, sr=sr, n_mfcc=13, n_fft=640, hop_length=320)
    mfcc = mfcc.T  # (time, n_mfcc)
    # Pad or truncate to fixed length (124 time steps)
    if mfcc.shape[0] < 124:
        pad_width = 124 - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')
    else:
        mfcc = mfcc[:124, :]
    return mfcc


In [ ]:
import tensorflow as tf

# Load model
model = tf.keras.models.load_model("/content/keyword_model.h5")

# Predict on each converted WAV file
for m4a_file in uploaded.keys():
    if m4a_file.endswith('.m4a'):
        wav_file = m4a_file.replace('.m4a', '.wav')
        mfcc = extract_mfcc(wav_file)
        input_tensor = np.expand_dims(mfcc, axis=0)  # Shape: (1, 124, 13)
        prediction = model.predict(input_tensor)
        predicted_label = np.argmax(prediction)
        print(f"{m4a_file} → Predicted label index: {predicted_label}, probabilities: {prediction}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
New Recording 56.m4a → Predicted label index: 0, probabilities: [[9.9999833e-01 1.6468498e-06 4.9509487e-11 4.3942507e-08]]


<ipython-input-71-846fff85ae75>:6: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal, sr=sr, n_mfcc=13, n_fft=640, hop_length=320)


In [ ]:
import os

input_file = "New Recording 56.m4a"
output_file = "New_Recording_56.wav"

# Convert using ffmpeg
!ffmpeg -y -i "$input_file" "$output_file"


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from IPython.display import Audio

# Play the converted file
Audio("New_Recording_56.wav")


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving New Recording 36.m4a to New Recording 36.m4a


In [ ]:
import os

# Convert all uploaded .m4a to .wav
for m4a_file in uploaded.keys():
    if m4a_file.endswith('.m4a'):
        wav_file = m4a_file.replace('.m4a', '.wav')
        os.system(f"ffmpeg -i '{m4a_file}' -ar 16000 -ac 1 '{wav_file}'")
        print(f"Converted {m4a_file} → {wav_file}")
import numpy as np
import librosa

def extract_mfcc(filepath):
    signal, sr = librosa.load(filepath, sr=16000)
    mfcc = librosa.feature.mfcc(signal, sr=sr, n_mfcc=13, n_fft=640, hop_length=320)
    mfcc = mfcc.T  # (time, n_mfcc)
    # Pad or truncate to fixed length (124 time steps)
    if mfcc.shape[0] < 124:
        pad_width = 124 - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')
    else:
        mfcc = mfcc[:124, :]
    return mfcc
import tensorflow as tf

# Load model
model = tf.keras.models.load_model("/content/keyword_model.h5")

# Predict on each converted WAV file
for m4a_file in uploaded.keys():
    if m4a_file.endswith('.m4a'):
        wav_file = m4a_file.replace('.m4a', '.wav')
        mfcc = extract_mfcc(wav_file)
        input_tensor = np.expand_dims(mfcc, axis=0)  # Shape: (1, 124, 13)
        prediction = model.predict(input_tensor)
        predicted_label = np.argmax(prediction)
        print(f"{m4a_file} → Predicted label index: {predicted_label}, probabilities: {prediction}")


Converted New Recording 36.m4a → New Recording 36.wav


<ipython-input-77-c8e67d7e167d>:14: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal, sr=sr, n_mfcc=13, n_fft=640, hop_length=320)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
New Recording 36.m4a → Predicted label index: 1, probabilities: [[3.4622738e-07 9.9997616e-01 9.6677180e-11 2.3479612e-05]]


In [ ]:
import os

input_file = "New Recording 36.m4a"
output_file = "New_Recording_36.wav"

# Convert using ffmpeg
!ffmpeg -y -i "$input_file" "$output_file"
from IPython.display import Audio

# Play the converted file
Audio("New_Recording_36.wav")


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab